In [2]:
import pandas as pd
import random as rnd 
import mysql.connector
from mysql.connector import Error
import plotly
import json
import csv
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pymysql
import plotly.express as px
import plotly.graph_objects as go
from sqlalchemy import create_engine
import networkx as nx
from networkx_viewer import Viewer

# Scales and Abbreviations Used for the Starter and Change Analyses 

In [10]:
info_dict = {'descriptor_domain':['abilites','work_context','interests',
                               'knowledge','skills','work_styles','work_values','gwas','job_zones'],
            'starter':['IM,LV',None,'OI',None,None,'IM','EX',None,None],
            'change':[None,'CX',None,'IM, LV','IM, LV',None,None,'IM, LV','JZ']}
df_info = pd.DataFrame(data = info_dict)
df_info

,descriptor_domain,starter,change
0,abilites,"IM,LV",None
1,work_context,None,CX
2,interests,OI,None
3,knowledge,None,"IM, LV"
4,skills,None,"IM, LV"
5,work_styles,IM,None
6,work_values,EX,None
7,gwas,None,"IM, LV"
8,job_zones,None,JZ


In [11]:
connection = mysql.connector.connect(host='localhost',
                                         database='employment',
                                         user='root',
                                         password='1234')

In [ ]:
def get_data(connection, descriptor_domain):
    '''
    Return df in the SOCs-decriptor format of a specific domain 
    '''
    df= pd.read_sql('''
        SELECT * FROM employment.'''+ descriptor_domain + ''';''')